In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
import joblib
import wandb

from scipy import sparse

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, DataLoader


from src.wandb_utils import WandbLogger, create_sweep_config
from src.models import KmerSequenceModel
from src.utils import load_kmer_counts, svm_project_kmer_vec, load_svd
from src.data import DNASequenceDataset

In [ ]:
# Code to Generate Vector Dataset
from src.data import KmerTokenizer

# 1KB Dataset
# df = pd.read_csv("../data/data_transformation/chipseq/updated_histone_mods_and_GA_factors.feature_matrix.1kb_res.txt", sep="\t")
# df['seq'] = df['seq'].str.upper()

# tokenizer = KmerTokenizer(k=8)
# df['kmer_vec'] = df['seq'].map(lambda x: tokenizer.seq_to_vec(x))

# full_dataset = sparse.vstack(df['kmer_vec'])
# sparse.save_npz("../data/sparse_kmer_counts.npz", full_dataset)

In [ ]:
 # uses default path ../data/sparse_kmer_counts.npz
full_dataset = load_kmer_counts(drop_unk=True)

### Fit SVM Models
TODO: need to replicate this for 64, 256 components

In [ ]:
svd_model = TruncatedSVD(n_components=128)
svd_model.fit(full_dataset)

joblib.dump(svd_model, "../models/seq_svm/svd_model_128.joblib")

In [ ]:
# Load fit SVM and Project KMer Counts Vectors
svd_128 = load_svd(128)
projected_vecs = svm_project_kmer_vec(
    svd_model=svd_128,
    full_dataset=full_dataset,
    batch_size=128
)

In [ ]:
# Create DNA Sequence Dataset from projection
sequences = df['seq']
gene_labels = df['gene_labels'].to_numpy()
mre_labels = df['mre_labels'].to_numpy()

metadata =  df.drop(columns=['seq', 'gene_labels', 'mre_labels'])

final_dataset = DNASequenceDataset(
    sequences, gene_labels, mre_labels, projected_vecs, metadata
)

torch.save(final_dataset, '../data/128_vec_dataset.pt')

# TRAINING SEGMENT

In [3]:
from sklearn.metrics import roc_auc_score, average_precision_score
from torcheval.metrics import MulticlassAUROC, MulticlassAUPRC, MulticlassAccuracy
import json

def get_input_gradients(model, criterion, test_loader, device, task, num_classes=None):
    model.eval()
    all_loss_grads   = []
    all_class_grads  = []

    for gene, mre, vecs in test_loader:
        labels = (mre if task == "mre" else gene).long().to(device)
        vecs   = vecs.to(device)
        vecs.requires_grad_(True)

        outputs = model(vecs)

        # per‐class gradients
        # for each class c we do d(outputs[:,c].sum)/d(vecs)
        class_grads = []
        C = outputs.shape[1]
        for c in range(C):
            grad_c = torch.autograd.grad(
                outputs[:, c].sum(), vecs, retain_graph=True
            )[0]
            class_grads.append(grad_c.detach().cpu())

        # stack into (B, C, D)
        class_grads = torch.stack(class_grads, dim=1)

        loss = criterion(outputs, labels)
        loss_grads = torch.autograd.grad(
            loss, vecs
        )[0].detach().cpu()

        all_loss_grads.append(loss_grads)
        all_class_grads.append(class_grads)

    # concat over batches
    loss_grads  = torch.cat(all_loss_grads,  dim=0)   # (N, D)
    class_grads = torch.cat(all_class_grads, dim=0)   # (N, C, D)

    return loss_grads, class_grads
    
def evaluate_model(model, criterion, test_loader, device, metrics, task, config_metrics, return_full=False):
    model.eval()
    for m in metrics:
        m.to(device)
        m.reset()
    val_loss = 0
    
    lab_pred_log = ([], [], []) if return_full else None
        
    with torch.no_grad():
        for gene, mre, vecs in test_loader:
            labels = mre if task == "mre" else gene
            labels = labels.long().to(device)
            vecs = vecs.to(device)

            outs = model(vecs)
            loss = criterion(outs, labels)
            val_loss += loss.detach().item()
            logits = F.softmax(outs, dim=1)
            preds = torch.argmax(logits, dim=1)
            
            for append, ls in zip([labels, preds, logits], lab_pred_log):
                ls.append(append.detach().cpu())
            
            for metric in metrics:
                metric.update(logits, labels)
    
    lab_pred_log = [torch.cat(ls) for ls in lab_pred_log] if return_full else None
    return val_loss, {metric_name: metric.compute().item() for metric, metric_name in zip(metrics, config_metrics)}, lab_pred_log

def train_model(model, criterion, optimizer, loaders, val_metrics, config):

    _NAME = f"{config.name}_{config.task}_{config.lr}_{config.wd}_weighted_{config.weighted}"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    patience = 0
    best_metric = 0
    train_acc = MulticlassAccuracy().to(device)
    best_metrics = {}
    
    for epoch in range(config.epochs):
        if patience == 5:
            break
        print(f"Epoch {epoch}:")
        train_loss = 0
        model.train()
        for gene, mre, vecs in loaders["train"]:            
            labels = mre if config.task == "mre" else gene
            labels = labels.long().to(device)
            vecs = vecs.to(device)

            optimizer.zero_grad()
            outs = model(vecs)
            loss = criterion(outs, labels)
            loss.backward()
            optimizer.step()

            logits = F.softmax(outs, dim=1)
            train_loss += loss.detach().item()
            train_acc.update(logits, labels)

        train_accuracy = train_acc.compute().item()
        print(f"Train Metrics:\n\tLoss:{train_loss:.4f}, Acc: {train_accuracy:.4f}:")
        train_acc.reset()
        
        # Compute Validation Scores and Determine Patience
        val_loss, metrics, _ = evaluate_model(
            model, criterion, loaders["test"], device, val_metrics, config.task, config.metrics
        )
        print("finished evaluation")

        wandb.log({"train_loss": train_loss, "train_accuracy": train_accuracy})
        wandb.log({"val_loss": val_loss})
        print(config.metrics)
        wandb.log({f"val_{metric_name}": metrics[metric_name] for metric_name in config.metrics})

        if metrics[config.target] > best_metric:
            patience = 0
            best_metric = metrics[config.target]
            best_metrics = metrics
            torch.save(model.state_dict(), f"{config.outdir}/{_NAME}_best.pt")
            
            metrics["epoch"] = epoch
            with open(f"{config.outdir}/{_NAME}_best_metrics.json", "w") as f:
                json.dump(metrics, f)
        else:
            patience += 1
        torch.save(model.state_dict(), f"{config.outdir}/{_NAME}_current.pt")
        
        # Logging!
        print("Validation Metrics:")
        print(f"\tLoss: {val_loss:.4f}")
        for name, metric in metrics.items():
            print(f"\t{name}: {metric:.4f}")
    
    best_model = torch.load(f"{config.outdir}/{_NAME}_best.pt", map_location=device)
    model.load_state_dict(best_model)
    model.to(device)
    return model, best_metrics

In [21]:
import wandb
wandb.login()

True

In [4]:
def get_data():
    # instance of kmer_data.DNASequenceDataset (contains sequences, projected vectors)
    data = torch.load('data_transformation/128_vec_dataset.pt')
    # ignore these two lines -- class counts. 
    _, mre_class_counts = np.unique(data.mre_labels, return_counts=True)
    _, gene_class_counts = np.unique(data.gene_labels, return_counts=True)

    mre_class_weights = torch.Tensor(1 / (mre_class_counts / np.sum(mre_class_counts)))
    gene_class_weights = torch.Tensor(1 / (gene_class_counts / np.sum(gene_class_counts)))

    train_size = int(0.8 * len(data))
    test_size = len(data) - train_size

    train_dataset, test_dataset = random_split(data, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    loaders = {"train": train_loader, "test": test_loader}

    return loaders, mre_class_weights, gene_class_weights

loaders, mre_class_weights, gene_class_weights = get_data()

def train():
    wandb.init(project="BindGPS", name="kmer_128")
    config = wandb.config
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if config.name == "kmer_128":
        model = KmerSequenceModel(128, hidden_dims=config.hidden_dims, num_classes=3)
        model.to(device)

    if config.weighted:
        weights = mre_class_weights if config.task == "mre" else gene_class_weights
        weights = weights.float().to(device)
        criterion = nn.CrossEntropyLoss(weight=weights)
    else:
        criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr, weight_decay=config.wd)

    val_metrics = [
        MulticlassAUROC(num_classes=3),
        MulticlassAUPRC(num_classes=3),
        MulticlassAccuracy(),
    ]

    loaders = {"train": train_loader, "test": test_loader}

    _, metrics = train_model(
        model,
        criterion,
        optimizer,
        loaders,
        val_metrics,
        config,
    )
    wandb.finish()


sweep_config = {
    "method": "grid",
    "metric": {
        "name": "auroc",
        "goal": "maximize"
    },
    "parameters": {
        "name": {"values": ["kmer_128"]},
        "hidden_dims": {"values": [(256, 128), (128, 64)]}, #, 
        "lr": {"values": [1e-3, 1e-4, 1e-5]},
        "wd": {"values": [1e-5, 1e-6]},
        "epochs": {"values": [10]},
        "seed": {"values": [42]},
        "weighted": {"values": [True, False]},
        "target": {"values": ["auroc"]},
        "task": {"values": ["mre", "gene"]},
        "metrics": {"values": [["auroc", "auprc", "accuracy"]]},
        "outdir": {"values": ["models/kmer_128"]}
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_config, project="bind_gps")
wandb.agent(sweep_id, function=train, count=100)

Create sweep with ID: ms3id6yy
Sweep URL: https://wandb.ai/pranavmahabs/bind_gps/sweeps/ms3id6yy


wandb: Agent Starting Run: txt5b9xc with config:
wandb: 	epochs: 10
wandb: 	hidden_dims: [256, 128]
wandb: 	lr: 0.001
wandb: 	metrics: ['auroc', 'auprc', 'accuracy']
wandb: 	name: kmer_128
wandb: 	outdir: models/kmer_128
wandb: 	seed: 42
wandb: 	target: auroc
wandb: 	task: mre
wandb: 	wd: 1e-05
wandb: 	weighted: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 0:
Train Metrics:
	Loss:1593.3532, Acc: 0.6846:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 374.7584
	auroc: 0.8240
	auprc: 0.5552
	accuracy: 0.7147
	epoch: 0.0000
Epoch 1:
Train Metrics:
	Loss:1484.3976, Acc: 0.7330:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 364.5660
	auroc: 0.8154
	auprc: 0.5303
	accuracy: 0.7058
Epoch 2:
Train Metrics:
	Loss:1444.6294, Acc: 0.7421:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 354.1389
	auroc: 0.8388
	auprc: 0.5610
	accuracy: 0.7352
	epoch: 2.0000
Epoch 3:
Train Metrics:
	Loss:1421.9388, Acc: 0.7483:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 347.5382
	auroc: 0.8377
	auprc: 0.5583
	accuracy: 0.7719
Epoch 4:
Train Metrics:
	Loss:1402.7707, Acc: 0.7537:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 354.8484
	auroc: 0.8388
	auprc: 0.5523
	accuracy: 0.7251
Epoch 5:
Train Metrics:
	

train_accuracy,▁▅▆▇▇█████
train_loss,█▅▄▃▃▂▂▂▁▁
val_accuracy,▃▂▅█▄▂▇▅▁▂
val_auprc,▆▁▇▆▅▆▇█▃▅
val_auroc,▃▁▇▇▇▇▇███
val_loss,█▅▃▁▃▃▂▂▂▂
train_accuracy,0.75898
train_loss,1338.56299
val_accuracy,0.70489
val_auprc,0.55098
val_auroc,0.84155


wandb: Agent Starting Run: 8k9syyoe with config:
wandb: 	epochs: 10
wandb: 	hidden_dims: [256, 128]
wandb: 	lr: 0.001
wandb: 	metrics: ['auroc', 'auprc', 'accuracy']
wandb: 	name: kmer_128
wandb: 	outdir: models/kmer_128
wandb: 	seed: 42
wandb: 	target: auroc
wandb: 	task: mre
wandb: 	wd: 1e-05
wandb: 	weighted: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Agent Starting Run: f8qvtwa7 with config:
wandb: 	epochs: 10
wandb: 	hidden_dims: [128, 64]
wandb: 	lr: 1e-05
wandb: 	metrics: ['auroc', 'auprc', 'accuracy']
wandb: 	name: kmer_128
wandb: 	outdir: models/kmer_128
wandb: 	seed: 42
wandb: 	target: auroc
wandb: 	task: gene
wandb: 	wd: 1e-05
wandb: 	weighted: True
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 0:
Train Metrics:
	Loss:1959.2321, Acc: 0.2661:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 467.9266
	auroc: 0.5542
	auprc: 0.3721
	accuracy: 0.2451
	epoch: 0.0000
Epoch 1:
Train Metrics:
	Loss:1891.5797, Acc: 0.3429:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 462.5811
	auroc: 0.5906
	auprc: 0.3978
	accuracy: 0.3242
	epoch: 1.0000
Epoch 2:
Train Metrics:
	Loss:1854.0585, Acc: 0.3912:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 460.4324
	auroc: 0.6096
	auprc: 0.4135
	accuracy: 0.3230
	epoch: 2.0000
Epoch 3:
Train Metrics:
	Loss:1834.8691, Acc: 0.4153:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 457.7049
	auroc: 0.6240
	auprc: 0.4229
	accuracy: 0.3300
	epoch: 3.0000
Epoch 4:
Train Metrics:
	Loss:1818.8286, Acc: 0.4320:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 457.4275
	auroc: 0.6341
	auprc: 0.4322
	accuracy: 0.

train_accuracy,▁▄▅▆▇▇▇███
train_loss,█▆▄▃▃▂▂▂▁▁
val_accuracy,▁▆▆▆▆▆▆▆██
val_auprc,▁▃▄▅▆▆▇▇██
val_auroc,▁▃▅▆▆▇▇███
val_loss,█▆▅▄▄▃▃▃▁▁
train_accuracy,0.46927
train_loss,1768.92842
val_accuracy,0.36533
val_auprc,0.45696
val_auroc,0.66057


wandb: Agent Starting Run: xn60z2vg with config:
wandb: 	epochs: 10
wandb: 	hidden_dims: [128, 64]
wandb: 	lr: 1e-05
wandb: 	metrics: ['auroc', 'auprc', 'accuracy']
wandb: 	name: kmer_128
wandb: 	outdir: models/kmer_128
wandb: 	seed: 42
wandb: 	target: auroc
wandb: 	task: gene
wandb: 	wd: 1e-05
wandb: 	weighted: False
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Epoch 0:
Train Metrics:
	Loss:1885.3849, Acc: 0.3783:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 455.8679
	auroc: 0.5749
	auprc: 0.3829
	accuracy: 0.4442
	epoch: 0.0000
Epoch 1:
Train Metrics:
	Loss:1684.3171, Acc: 0.5357:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 419.1891
	auroc: 0.6016
	auprc: 0.4030
	accuracy: 0.5942
	epoch: 1.0000
Epoch 2:
Train Metrics:
	Loss:1590.9755, Acc: 0.5799:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 400.5698
	auroc: 0.6299
	auprc: 0.4243
	accuracy: 0.5936
	epoch: 2.0000
Epoch 3:
Train Metrics:
	Loss:1550.0063, Acc: 0.5883:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 389.5472
	auroc: 0.6345
	auprc: 0.4261
	accuracy: 0.5945
	epoch: 3.0000
Epoch 4:
Train Metrics:
	Loss:1529.5367, Acc: 0.5924:
finished evaluation
['auroc', 'auprc', 'accuracy']
Validation Metrics:
	Loss: 386.0733
	auroc: 0.6440
	auprc: 0.4331
	accuracy: 0.

# EVALUATION

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaders, mre_class_weights, gene_class_weights = get_data()

# Load best MRE model, best GENE model
model = KmerSequenceModel(128, hidden_dims=(128, 64), num_classes=3)
best_model = torch.load("models/kmer_128/kmer_128_mre_0.001_1e-06_weighted_False_best.pt", map_location=device)
model.load_state_dict(best_model)
mre_model = model.to(device)

# Initialize Other Items
val_metrics = [
    MulticlassAUROC(num_classes=3),
    MulticlassAUPRC(num_classes=3),
    MulticlassAccuracy(),
]
config_metrics = ["auroc", "auprc", "accuracy"]
task = "mre"
criterion = nn.CrossEntropyLoss()

val_loss, metrics, full = evaluate_model(
    model, criterion, loaders["test"], device, val_metrics, task, config_metrics, return_full=True, input_grads=False
)

In [22]:
loss_grads, class_grads = get_input_gradients(
    model, criterion, loaders["test"], device, task
)

In [50]:
filename = 'models/svd_model_128.joblib'
svd_model = joblib.load(filename)
comps = torch.from_numpy(svd_model.components_).to(device)

In [38]:
from torch.utils.data import TensorDataset, DataLoader
agg_loss   = torch.zeros(comps.size()[1],device=device)
agg_class  = torch.zeros(3, comps.size()[1], device=device)

batch_size = 64
ds = TensorDataset(loss_grads, class_grads)
loader = DataLoader(ds, batch_size=batch_size, shuffle=False)
comps = comps.to(torch.float32)

for loss_batch, class_batch in loader:
    loss_batch, class_batch = loss_batch.to(device), class_batch.to(device)
    loss_backproj  = loss_batch @ comps
    class_backproj = torch.matmul(class_batch, comps)
    
    agg_loss  += loss_backproj.abs().sum(dim=0)
    agg_class += class_backproj.abs().sum(dim=0)

In [57]:
top_overall = list(torch.topk(agg_loss,k=15).indices.cpu().numpy())
print(top_overall)
top_indices = []
for i in range(3):
    class_proj = agg_class[i].squeeze()
    top_indices.append(list(torch.topk(class_proj,k=15).indices.cpu().numpy()))

[8738, 30583, 34952, 56797, 2184, 56799, 32904, 546, 30591, 34953, 56829, 8736, 40413, 8226, 24029]


In [59]:
kmer_tokenizer = KmerTokenizer(k=8)

print("OVERALL TOP 8-mers")
print(kmer_tokenizer.detokenize(top_overall))

print("\nClass Specific Top 8-mers")
for i in range(3):
    print(f"Class {i}: {kmer_tokenizer.detokenize(top_indices[i])}")

OVERALL TOP 8-mers
['AGAGAGAG', 'CTCTCTCT', 'GAGAGAGA', 'TCTCTCTC', 'AAGAGAGA', 'TCTCTCTT', 'GAAAGAGA', 'AAAGAGAG', 'CTCTCTTT', 'GAGAGAGC', 'TCTCTTTC', 'AGAGAGAA', 'GCTCTCTC', 'AGAAAGAG', 'CCTCTCTC']

Class Specific Top 8-mers
Class 0: ['AGAGAGAG', 'CTCTCTCT', 'GAGAGAGA', 'TCTCTCTC', 'AAGAGAGA', 'TCTCTCTT', 'GAAAGAGA', 'AAAGAGAG', 'CTCTCTTT', 'GAGAGAGC', 'AGAGAGAA', 'TCTCTTTC', 'GCTCTCTC', 'AGAAAGAG', 'CCTCTCTC']
Class 1: ['AGAGAGAG', 'CTCTCTCT', 'GAGAGAGA', 'TCTCTCTC', 'AAGAGAGA', 'TCTCTCTT', 'GAAAGAGA', 'CTCTCTTT', 'AAAGAGAG', 'AGAGAGAA', 'TCTCTTTC', 'GAGAGAGC', 'GCTCTCTC', 'AGAAAGAG', 'CCTCTCTC']
Class 2: ['AGAGAGAG', 'CTCTCTCT', 'GAGAGAGA', 'TCTCTCTC', 'AAGAGAGA', 'TCTCTCTT', 'GAAAGAGA', 'AAAGAGAG', 'CTCTCTTT', 'TCTCTTTC', 'GAGAGAGC', 'AGAGAGAA', 'GCTCTCTC', 'AGAAAGAG', 'TTCTCTCT']


# Gene Task Evaluation

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaders, mre_class_weights, gene_class_weights = get_data()

# Load best MRE model, best GENE model
model = KmerSequenceModel(128, hidden_dims=(128, 64), num_classes=3)
best_model = torch.load("models/kmer_128/kmer_128_gene_0.001_1e-05_weighted_False_best.pt", map_location=device)
model.load_state_dict(best_model)
mre_model = model.to(device)

# Initialize Other Items
val_metrics = [
    MulticlassAUROC(num_classes=3),
    MulticlassAUPRC(num_classes=3),
    MulticlassAccuracy(),
]
config_metrics = ["auroc", "auprc", "accuracy"]
task = "gene"
criterion = nn.CrossEntropyLoss()

val_loss, metrics, full = evaluate_model(
    model, criterion, loaders["test"], device, val_metrics, task, config_metrics, return_full=True
)

In [9]:
loss_grads, class_grads = get_input_gradients(
    model, criterion, loaders["test"], device, task
)
filename = 'models/svd_model_128.joblib'
svd_model = joblib.load(filename)
comps = torch.from_numpy(svd_model.components_).to(device)

In [10]:
from torch.utils.data import TensorDataset, DataLoader
agg_loss   = torch.zeros(comps.size()[1],device=device)
agg_class  = torch.zeros(3, comps.size()[1], device=device)

batch_size = 64
ds = TensorDataset(loss_grads, class_grads)
loader = DataLoader(ds, batch_size=batch_size, shuffle=False)
comps = comps.to(torch.float32)

for loss_batch, class_batch in loader:
    loss_batch, class_batch = loss_batch.to(device), class_batch.to(device)
    loss_backproj  = loss_batch @ comps
    class_backproj = torch.matmul(class_batch, comps)
    
    agg_loss  += loss_backproj.abs().sum(dim=0)
    agg_class += class_backproj.abs().sum(dim=0)

In [11]:
top_overall = list(torch.topk(agg_loss,k=15).indices.cpu().numpy())
print(top_overall)
top_indices = []
for i in range(3):
    class_proj = agg_class[i].squeeze()
    top_indices.append(list(torch.topk(class_proj,k=15).indices.cpu().numpy()))
    
kmer_tokenizer = KmerTokenizer(k=8)

print("OVERALL TOP 8-mers")
print(kmer_tokenizer.detokenize(top_overall))

print("\nClass Specific Top 8-mers")
for i in range(3):
    print(f"Class {i}: {kmer_tokenizer.detokenize(top_indices[i])}")

[43690, 21845, 8738, 34952, 30583, 56797, 15163, 60652, 52942, 0, 20805, 17684, 16371, 65023, 256]
OVERALL TOP 8-mers
['GGGGGGGG', 'CCCCCCCC', 'AGAGAGAG', 'GAGAGAGA', 'CTCTCTCT', 'TCTCTCTC', 'ATGTATGT', 'TGTATGTA', 'TATGTATG', 'AAAAAAAA', 'CCACCACC', 'CACCACCA', 'ATTTTTAT', 'TTTCTTTT', 'AAACAAAA']

Class Specific Top 8-mers
Class 0: ['AGAGAGAG', 'GAGAGAGA', 'GGGGGGGG', 'CCCCCCCC', 'CTCTCTCT', 'TCTCTCTC', 'ATGTATGT', 'TGTATGTA', 'TATGTATG', 'CCACCACC', 'CACCACCA', 'AAACAAAA', 'TTTGTTTT', 'ATTTTTAT', 'TTTCTTTT']
Class 1: ['GGGGGGGG', 'CCCCCCCC', 'AGAGAGAG', 'GAGAGAGA', 'AAAAAAAA', 'AAACAAAA', 'CCACCACC', 'GCTGCTGC', 'TTTGTTTT', 'CACCACCA', 'ATCTTATC', 'TCTTATCT', 'TATCTTAT', 'TTTTTTTT', 'TGCTGCTG']
Class 2: ['CCCCCCCC', 'GGGGGGGG', 'TTTTTTTT', 'AAAAAAAA', 'AGAGAGAG', 'CTCTCTCT', 'GAGAGAGA', 'TCTCTCTC', 'TATGTATG', 'ATGTATGT', 'TGTATGTA', 'CAAAAAAA', 'TATTATTA', 'TATAAATT', 'ATTATTAT']
